In [ ]:
import tensorflow as tf
import pandas as pd
import json
import configparser
from pathlib import Path
import os
    
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN

from sklearn.metrics import mean_absolute_percentage_error as MAPE

configs = configparser.ConfigParser()
configs.read("../configuracoes.conf",encoding='utf-8')

path = f"data/accb_custo_total_ilheus.xlsx"
look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses previstos

In [ ]:
def create_seq(data):
    for n_step in range(1, look_back + forecast_horizon):
        data.loc[::, f'preco t(h + {n_step})'] = data['preco'].shift(-n_step).values
    data.dropna(inplace=True)
    data.reset_index(drop=True, inplace=True)
    
    return data

number_iter = 3
cross_validate_mapes = [[] for i in range(number_iter)]

for j in range(number_iter):
    i = 0
    # definir i < 42 para validação e i < 48 para teste
    while i < 48:

        # Carrega os dados
        cesta = pd.read_excel(path)
        cesta.drop(['ano'], axis=1, inplace=True)
        cesta = cesta / 1000
        #cesta = cesta.iloc[:-9:]

        # Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
        row_limit = 142 # 142 para teste e 100 para validação
        
        # Divide os dados em Treino e Teste
        train = cesta.iloc[:row_limit + i:]
        test = cesta.iloc[row_limit + i::] 
        train = create_seq(train.copy())
        test = create_seq(test.copy())
        
        # Divide os dados em features e targets
        X_train = train.iloc[::, :look_back:].values
        y_train = train.iloc[::, look_back::].values
        X_test = test.iloc[: 6:, :look_back:].values
        y_test = test.iloc[: 6:, look_back::].values
        y_test2 = test.iloc[: 6:, look_back::].values.tolist()

        i += 6
        #remodela os dados para formato (batches, sequencia de tempo, features)
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        #Cria o modelo
        model=Sequential()
        model.add(SimpleRNN(24, stateful=True, input_shape=(look_back, 1), batch_input_shape=(1, look_back, 1)))
        model.add(Dense(forecast_horizon))
        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

        #Treina e prever (forecast_horizon) quantidade de tempo
        history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
        forecast = model.predict(X_test, batch_size=1)

        cross_validate_mapes[j].append(MAPE(y_test2, forecast) * 100)

In [ ]:
print("cross_validate_mapes: ", cross_validate_mapes)
cross_validate_means = []

for current_mapes in cross_validate_mapes.copy():
    mean = 0
    for mape in current_mapes:
        mean += mape

    mean /= len(current_mapes)
    cross_validate_means.append(mean)

print("cross_validate_means: ", cross_validate_means)

if len(cross_validate_means) >= 3:
    cross_validate_means.sort()
    cross_validate_means.pop()
    del cross_validate_means[0]

general_mean = 0
for mean in cross_validate_means:
    general_mean += mean
general_mean /= len(cross_validate_means)

print("MAPE = ", round(general_mean, 3))